## Testes com variáveis de ambiente no pydantic

In [1]:
import glob
import os
import pathlib

from pydantic import Field, SecretStr, ValidationError
from pydantic_settings import BaseSettings, SettingsConfigDict

env_file = [f for f in pathlib.Path().glob("*.env")]
with open(env_file[0]) as f:
    print(f.read())

os.environ["K8S_SECRET_CREDENTIALS"] = "dummy"
os.environ["K8S_SECRET_GOOGLE_AUTH_KEY"] = "dummy_key"
os.environ["ENVIRONMENT"] = "prod"
os.environ["REDIS_DB"] = "1"
# os.environ["NFS_BACKUP_PATH"] = "dummy"


class KubectlSecrets(BaseSettings):
    credentials: str
    google_auth_key: str

    model_config = SettingsConfigDict(env_prefix="K8S_SECRET_")


class NFSConfig(BaseSettings):
    """Configurações do NFS"""

    nfs_backup_path: str = Field(..., env="NFS_BACKUP_PATH")

    model_config = SettingsConfigDict(env_prefix="", env_file=env_file, extra="allow")


class AWSConfig(BaseSettings):
    """Configurações do AWS"""

    aws_access_key_id: str = Field(..., env="AWS_ACCESS_KEY_ID")
    aws_secret_access_key: SecretStr = Field(..., env="AWS_SECRET_ACCESS_KEY")
    aws_bucket_name: str = Field(..., env="AWS_BUCKET_NAME")

    model_config = SettingsConfigDict(
        env_prefix="AWS_", env_file=env_file, extra="allow"
    )


class Settings(BaseSettings):
    kubectl_secrets: KubectlSecrets = KubectlSecrets()
    nfs: NFSConfig = NFSConfig()
    aws: AWSConfig = AWSConfig()
    environment: str = "development"
    redis_db: str = "0"

    model_config = SettingsConfigDict(env_prefix="", extra="allow")


try:
    settings = Settings()

    print(settings.nfs.nfs_backup_path)
    print(settings.aws.aws_access_key_id)
except ValidationError as exc:
    print(repr(exc.errors()[0]["type"]))
    # > 'extra_forbidden'


IndexError: list index out of range

## Lidando com exceptions

In [ ]:
from pydantic import BaseModel, ConfigDict, ValidationError


class Model(BaseModel):
    x: str
    y: str

    model_config = ConfigDict(extra="forbid")


try:
    model = Model(x="test", y="test")
    print(f"{model.x}")
except ValidationError as exc:
    print(repr(exc.errors()[0]["type"]))
    # > 'extra_forbidden'


## Carregando variáveis

In [ ]:
# from pydantic import Field, SecretStr
# from pydantic_settings import BaseSettings

# class AWSConfig(BaseSettings):
#     aws_access_key_id: str = Field(..., env="AWS_ACCESS_KEY_ID")
#     aws_secret_access_key: SecretStr = Field(..., env="AWS_SECRET_ACCESS_KEY")
#     aws_bucket_name: str = Field(..., env="AWS_BUCKET_NAME")

#     class Config:
#         env_file = ".env"  # Carrega variáveis de um arquivo .env se necessário


# settings = AWSConfig()

import pathlib

# print(settings)
from pydantic import Field, SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict

env_file = f"{pathlib.Path(__file__).resolve().parent}/env"

print(env_file)

with open(env_file) as f:
    print(f.read())


class Settings(BaseSettings):
    app_name: str = "Awesome API"
    admin_email: str = Field(..., env="ADMIN_EMAIL")
    items_per_user: int = 50

    model_config = SettingsConfigDict(env_file=env_file)


class AWSConfig(BaseSettings):
    aws_access_key_id: str = Field(..., env="AWS_ACCESS_KEY_ID")
    aws_secret_access_key: SecretStr = Field(..., env="AWS_SECRET_ACCESS_KEY")
    aws_bucket_name: str = Field(..., env="AWS_BUCKET_NAME")

    class Config:
        env_file = f"{pathlib.Path(__file__).resolve().parent}/.env"
        # env_file = ".env"  # Carrega variáveis de um arquivo .env se necessário


settings = Settings()
settings_aws = AWSConfig()

print(settings_aws.aws_access_key_id)
print(settings.admin_email)


In [3]:
from pydantic_settings import BaseSettings
from pydantic import Field, SecretStr

import pathlib
import os

os.environ["AWS_ACCESS_KEY_ID"] = "dummy"
os.environ["AWS_SECRET_ACCESS_KEY"] = "dummy"
os.environ["AWS_BUCKET_NAME"] = "dummy"
os.environ["STORAGE_PROVIDERS"] = '["aws", "nfs","google"]'

#list_storage_providers = os.getenv("STORAGE_PROVIDERS").split(",")

# print(list_storage_providers)
print(os.getenv("STORAGE_PROVIDERS"))

class Config(BaseSettings):
    storage_providers: list = Field(default_factory=list, env="STORAGE_PROVIDERS")
    
confgig = Config()


class AWSConfig(BaseSettings):
    aws_access_key_id: str = Field(..., env="AWS_ACCESS_KEY_ID")
    aws_secret_access_key: SecretStr = Field(..., env="AWS_SECRET_ACCESS_KEY")
    aws_bucket_name: str = Field(..., env="AWS_BUCKET_NAME")

services = []

if  'aws' in confgig.storage_providers:
    services.append(AWSConfig())


for service in services:
    print(service.aws_secret_access_key)
    
# class AWSConfig(BaseSettings):
#     aws_access_key_id: str = Field(..., env="AWS_ACCESS_KEY_ID")
#     aws_secret_access_key: SecretStr = Field(..., env="AWS_SECRET_ACCESS_KEY")
#     aws_bucket_name: str = Field(..., env="AWS_BUCKET_NAME")

#     # class Config:
#     #     env_file = f"{pathlib.Path(__file__).resolve().parent}/.env"
#     #     # env_file = ".env"  # Carrega variáveis de um arquivo .env se è necessário 
    
#     self.services = [GoogleCloudService(config), NFSService(config), S3Service(config, logger), OracleCloudService(config)]
#     if "aws" in config.storage_providers:
#         self.services.redis_backupend(S3Service(config, logger))
#     if "oracle" in config.storage_providers:
#         self.services.redis_backupend(OracleCloudService(config))
#     if "google" in config.storage_providers:
#         self.services.redis_backupend(GoogleCloudService(config))
#     if "nfs" in config.storage_providers:
#         self.services.redis_backupend(NFSService(config))

print(os.getcwd())

["aws", "nfs","google"]
**********
/Users/francisconunes/Projects/redis_backup/scripts
